In [ ]:
#importing the libraries
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
import os
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()

In [ ]:
#checking we are getting access to TPU or not

%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.6.0
Running on TPU  ['10.126.40.98:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.126.40.98:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.126.40.98:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
#mounting the google drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing the dataset
path = "/content/drive/MyDrive//BreakHis_40x"
types = os.listdir(path)
print(types)
x = []
y = []

['Malignant', 'Benign']


In [ ]:
# Image pre-processing:
 
for file in types:
    typ= os.listdir(path+'//'+file)
    for image in typ[:]:
        image1 = cv2.imread(path+'//'+file+'//'+image)
        image1=cv2.resize(image1,(224,224))
        image1 = image1/255.0
        x.append(image1) 
        y.append(file)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

from sklearn.preprocessing import LabelEncoder
L = LabelEncoder()
y_train = L.fit_transform(y_train)
y_test = L.fit_transform(y_test)
y_val=L.fit_transform(y_val)

In [ ]:
#import the necessary libraries to build a necessary model
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K
from keras.optimizers import *
import numpy as np

In [ ]:
#creating a densenet model
def densenet(input_shape, n_classes, filters = 32):
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    def dense_block(x, repetition): 
        for _ in range(repetition):
            y1 = bn_rl_conv(x, 4*filters)
            y1 = bn_rl_conv(y1, filters, 3)
            x = concatenate([y1,x])
        return x 
    def transition_layer(x):
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x 
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    for repetition in [16]:   
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)  
    model = Model(input, output)
    return model

In [ ]:
#giving the input to the model
input_shape = 224, 224, 3
n_classes = 2
model = densenet(input_shape,n_classes)

In [ ]:
# compiling the model
model.compile(optimizer=keras.optimizers.Adam(lr=1e-5), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
#fitting the model
history=model.fit(np.array(X_train),y_train,epochs=20,validation_data=(np.array(X_val),y_val),batch_size=256)

Epoch 1/20
5/5 [==============================] - 147s 28s/step - loss: 0.5329 - accuracy: 0.7272 - val_loss: 0.6618 - val_accuracy: 0.6456
Epoch 2/20
5/5 [==============================] - 140s 28s/step - loss: 0.5127 - accuracy: 0.7542 - val_loss: 0.6585 - val_accuracy: 0.6456
Epoch 3/20
5/5 [==============================] - 140s 28s/step - loss: 0.4961 - accuracy: 0.7745 - val_loss: 0.6548 - val_accuracy: 0.6456
Epoch 4/20
5/5 [==============================] - 140s 28s/step - loss: 0.4856 - accuracy: 0.7922 - val_loss: 0.6519 - val_accuracy: 0.6456
Epoch 5/20
5/5 [==============================] - 140s 28s/step - loss: 0.4677 - accuracy: 0.8074 - val_loss: 0.6503 - val_accuracy: 0.6456
Epoch 6/20
5/5 [==============================] - 140s 28s/step - loss: 0.4578 - accuracy: 0.8108 - val_loss: 0.6505 - val_accuracy: 0.6456
Epoch 7/20
5/5 [==============================] - 139s 28s/step - loss: 0.4465 - accuracy: 0.8176 - val_loss: 0.6514 - val_accuracy: 0.6456
Epoch 8/20
5/5 [====

In [ ]:
#checking the accuracy
test_loss,test_acc=model.evaluate(np.array(X_test),y_test,verbose=5)

In [ ]:
test_acc

0.6860759258270264